## Customers Revenue Analytics

Customers of a company dbo.Users receive various offers dbo.Offers.

The company is interested in measuring the activity dbo.Activity resulting from communications dbo.Comms of these offers.

* I have used the notebook to get some overview of data and to calculate the Response rate for the offers.

* The average revenue by month and year graph can be used to filter and interact with the data in the dashboard.

#### SQL statement used for extracting the user responses to the offers:
    
SELECT * from dbo.Comms
LEFT JOIN dbo.Activity
ON dbo.Comms.UserId = dbo.Activity.UserId 
WHERE dbo.Comms.SendDate = dbo.Activity.[Date] 
OR dbo.Comms.SendDate = DATEADD(d,1,dbo.Activity.[Date]);

#### Condition used in Tableau to extract age from date of birth:

IF DATEADD('year', DATEDIFF('year',[DOB] , TODAY()), [DOB])> TODAY() THEN
    DATEDIFF('year', [DOB], TODAY())-1
ELSE
    DATEDIFF('year', [DOB], TODAY())
END

In [2]:
import pandas as pd

#### Importing the data sets/tabels downloaded from the provided database

In [3]:
activitydf = pd.read_csv('dbo.Activity.csv')
usersdf = pd.read_csv('dbo.Users.csv')
commsdf = pd.read_csv('dbo.Comms.csv')
offersdf = pd.read_csv('dbo.Offers.csv')

#### Basic overview of data

In [26]:
activitydf.head()

,UserId,Date,Revenue
0,1326865,19-02-2017,17.05
1,1326865,28-02-2017,17.05
2,1326865,06-03-2017,17.16
3,1326865,07-03-2017,17.16
4,1326865,08-03-2017,17.16


In [19]:
usersdf.head()

,UserId,Country,DOB,Gender,RegDate
0,1326865,DE,04-05-1988,M,24-03-2016
1,1333278,GB,21-07-1971,M,09-02-2016
2,1336967,GB,31-07-1983,M,29-01-2016
3,1339784,Other,21-09-1976,M,21-01-2016
4,1339829,Other,03-05-1981,F,12-04-2016


In [20]:
commsdf.head()

,UserId,SendDate,OfferId
0,1326865,29-11-2016,5
1,1326865,01-12-2016,4
2,1326865,07-12-2016,8
3,1326865,15-12-2016,9
4,1326865,19-12-2016,1


In [21]:
offersdf.head()

,Name,responserate
OfferId,,
1,Offer 1,3.66%
2,Offer 2,4.18%
3,Offer 3,3.28%
4,Offer 4,3.84%
5,Offer 5,3.51%


In [34]:
activitydf.duplicated().value_counts()

False    22642
dtype: int64

In [35]:
activitydf.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 22642 entries, 0 to 22641
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   UserId   22642 non-null  int64  
 1   Date     22642 non-null  object 
 2   Revenue  22642 non-null  float64
dtypes: float64(1), int64(1), object(1)
memory usage: 530.8+ KB


Importing the data obtained from running the above mentioned SQL statement to calculate the Response rate:

As I am using tableau for visualizing the results, I have left few of the duplicate fields in this data set and filterd them out out later in tableau.

In [12]:
responsedf = pd.read_csv('responses.csv')

In [13]:
responsedf.head()

,UserId,SendDate,OfferId,UserId.1,Date,Revenue
0,1336967,2016-12-13,3,1336967,2016-12-12,20.00
1,1339829,2017-02-19,1,1339829,2017-02-19,17.05
2,1417297,2017-04-16,8,1417297,2017-04-16,11.68
3,1610096,2017-09-07,10,1610096,2017-09-06,100.00
4,1617757,2017-03-13,9,1617757,2017-03-13,11.68


In [14]:
offersdf.set_index('OfferId', inplace=True)

#### Calculating response rate.

In [15]:
offersdf['responserate'] = (responsedf['OfferId'].value_counts().sort_index()
                            /commsdf['OfferId'].value_counts().sort_index()).apply("{0:.2%}".format)

In [16]:
offersdf

,Name,responserate
OfferId,,
1,Offer 1,3.66%
2,Offer 2,4.18%
3,Offer 3,3.28%
4,Offer 4,3.84%
5,Offer 5,3.51%
6,Offer 6,3.45%
7,Offer 7,3.14%
8,Offer 8,2.97%
9,Offer 9,3.38%


In [17]:
#Exporting the processed data for Visualization
offersdf.to_csv('Offers.csv')